In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data/heroes.csv", index_col=0)
df["Total"] = df.sum(axis=1)
# df[df["Total"] != 3]


In [ ]:
# df.to_csv("data/heroes.csv")

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import dash_bio

In [ ]:


fig = go.Figure(data=go.Heatmap(z=df.values, x=df.columns, y=df.index))
fig.show()


In [ ]:

fig_df = df.T.corr()
dash_bio.Clustergram(
    data=fig_df.values,
    column_labels=list(fig_df.columns.values),
    row_labels=list(fig_df.index.values),
    height=1200,
    width=1200,
    #     color_map = [
    #     [0.0, '#636EFA'],
    #     [0.25, '#AB63FA'],
    #     [0.5, '#FFFFFF'],
    #     [0.75, '#E763FA'],
    #     [1.0, '#EF553B']
    # ]
    
)

In [6]:

import pandas as pd
from ortools.linear_solver import pywraplp

def load_hero_token_data():
    return pd.read_csv("data/heroes.csv", index_col=0)

In [7]:
def integer_linear_solver(val_df, requirements, cost_col, optimization_problem_type="SAT"):
    """Solves the integer linear problem with the given requirements and cost column."""
    solver = pywraplp.Solver.CreateSolver(optimization_problem_type)
    solver.parameters.enumerate_all_solutions = True
    if not solver:
        return
    print("Solving with " + optimization_problem_type)

    infinity = solver.infinity()

    # Variables and constraints
    variables = []
    for idx, row in val_df.iterrows():
        variables.append(solver.IntVar(0, infinity, f"{idx}"))
    constraints = []
    for i, (token, required) in enumerate(requirements):
        constraints.append(solver.Constraint(required, infinity))
        for j, row in val_df.iterrows():
            # print(i, j, variables[val_df.index.get_loc(j)], row[token])
            constraints[i].SetCoefficient(
                var=variables[val_df.index.get_loc(j)], coeff=int(row[token])
            )

    # Objective and cost
    objective = solver.Objective()
    for i, row in val_df.iterrows():
        objective.SetCoefficient(
            var=variables[val_df.index.get_loc(i)],
            coeff=float(row[cost_col]),
        )
    objective.SetMinimization()

    # Solve
    print("Number of variables = %d" % solver.NumVariables())
    print("Number of constraints = %d" % solver.NumConstraints())

    result_status = solver.Solve()

    # The problem has an optimal solution.
    assert result_status == pywraplp.Solver.OPTIMAL

    # The solution looks legit (when using solvers others than
    # GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
    assert solver.VerifySolution(1e-7, True)

    print("Problem solved in %f milliseconds" % solver.wall_time())

    # The objective value of the solution.
    print("Optimal objective value = %f" % solver.Objective().Value())

    # The value of each variable in the solution.
    result_df = pd.DataFrame(columns=["Matches"])
    for variable in variables:
        if variable.solution_value() > 0:
            print("%s = %f" % (variable.name(), variable.solution_value()))
            # all the tokens from original dataframe + matches number from solution_value
            row = val_df.loc[variable.name()].copy()
            row["Matches"] = variable.solution_value()
            result_df = pd.concat([result_df, row.to_frame().T])

    print("Advanced usage:")
    print("Problem solved in %d branch-and-bound nodes" % solver.nodes())
    print("\n")
    return result_df

In [87]:
required_tokens = [
    ["Teleporting", 1.0],
    ["Walking", 1.0],
    ["Running", 1.0],
    ["Flying", 2.0]
]
df = load_hero_token_data()

In [99]:
df = df.sample(frac=1)
integer_linear_solver(df, required_tokens, "Playability")

Solving with SAT
Number of variables = 124
Number of constraints = 4
Problem solved in 39.000000 milliseconds
Optimal objective value = 3.000000
Meepo = 1.000000
Nightstalker = 2.000000
Advanced usage:
Problem solved in 0 branch-and-bound nodes




C:\Users\munti\AppData\Local\Temp\ipykernel_16732\4182805563.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, row.to_frame().T])


,Matches,Playability,Teleporting,Nuker,Floating,Healer,Walking,Mounted,Crawling,Running,...,Flying,Disabler,Durable,Initiator,Pusher,Jumping,Slithering,Escape,Melee,Total
Meepo,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0
Nightstalker,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
